In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
CPU times: user 156 ms, sys: 15.6 ms, total: 172 ms
Wall time: 202 ms


In [4]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text
Speed dropped from 1200 to 1000 it/s

In [5]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


def pre3(row) -> str:
    return mylib.pre3(row["text2"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)
col = "text3"
print(col)
df[col] = df.progress_apply(pre3, axis=1)

text1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14251/14251 [00:37<00:00, 381.07it/s]


text2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14251/14251 [04:32<00:00, 52.36it/s]


text3


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14251/14251 [02:30<00:00, 95.00it/s]


In [6]:
cols = ["text", "text1", "text2", "text3"]
df[cols].sample(10).head(10)

,text,text1,text2,text3
13704,"Incidently, the ignorant only ignores the truth.","Incidently, the ignorant only ignores the truth.","Incidently, the ignorant only ignores the truth.",incidently the ignorant only ignore the truth
6429,"Fantastic! Give me a block and blantantly ignore the behaviours of the others. So, it's controversal to state why i'm so irate, but yet it's within guidelines to:\n Insult my intelligence\n Degrade my character\n and overall tell me to fuck off?\n\nWow. 4:21 15 January 2012 (UTC)","Fantastic! Give me a block and blantantly ignore the behaviours of the others. So, it's controversal to state why i'm so irate, but yet it's within guidelines to: Insult my intelligence Degrade my character and overall tell me to fuck off? Wow. 4:21 15 January 2012 (UTC)","Fantastic! Give me a block and blantly ignore the behaviours of the others. So, it is controversal to state why I am so irate, but yet it is within guidelines to: Insult my intelligence Degrade my character and overall tell me to fuck off? Wow. 4:21 15 January 2012 (UTC)",fantastic give i a block and blantly ignore the behaviour of the other so it be controversal to state why i be so irate but yet it be within guideline to insult my intelligence degrade my character and overall tell i to fuck off wow 4 21 15 january 2012 utc
5922,"Plot Hole\nSTOP MESSING AROUND WITH THE PLOT HOLE PART!!! IT DESERVES TO BE THERE!!! SO STOP ERASING IT!!!Tim, 6 January, 2006","Plot Hole STOP MESSING AROUND WITH THE PLOT HOLE PART!!! IT DESERVES TO BE THERE!!! SO STOP ERASING IT!!!Tim, 6 January, 2006","Plot Hole STOP MESSING AROUND WITH THE PLOT HOLE PART!!! IT DESERVES TO BE THERE!!! SO STOP ERASING IT!!!Tim, 6 January, 2006",plot hole stop mess around with the plot hole part it deserve to be there so stop erase it tim 6 january 2006
5543,You must be logged in to upload files. \n\nSo In other words I have to sign up just to add 3 things to 1 entry.\n\nTo hell with that!!!!! This place can ROT for all I care!!!\n\n75.8.35.177,You must be logged in to upload files. So In other words I have to sign up just to add 3 things to 1 entry. To hell with that!!!!! This place can ROT for all I care!!!,You must be logged in to upload files. So In other words I have to sign up just to add 3 things to 1 entry. To hell with that!!!! This place can ROT for all I care!!!,you must be log in to upload file so in other word i have to sign up just to add 3 thing to 1 entry to hell with that this place can rot for all i care
8339,"Can you explain why you are needed? Excuse me, but removing thigns I wrote on my page would be the same as me removing things from YOUR user page. timecop","Can you explain why you are needed? Excuse me, but removing thigns I wrote on my page would be the same as me removing things from YOUR user page. timecop","Can you explain why you are needed? Excuse me, but removing thigns I wrote on my page would be the same as me removing things from YOUR user page. timecop",can you explain why you be need excuse i but remove thign i write on my page would be the same as i remove thing from your user page timecop
13584,"the truth \n\nPLease explain to me, sentence by sentence, why you have edited out the following addition:\n\nThe main impact zone of the crash was approximately 19 metres (57 feet) in width. No wreckage was found from the airliner within this impact zone or inside the building. The lawn immediately in front of the crash site was unmarked. Within the main impact zone was a hole that the object punched in the building, approximately 9 metres (27 feet) in width. The Pentagon is composed of five concentric rings. Only the outer three rings were penetrated. The upper floors of the first ring collapsed, but the structural integrity of the second and third rings remained. At the end of the third ring was a circular 'punch out' hole, again 9 metres (27 feet) in diameter. The wingspan of a Boeing 757 is 38 metres (114 feet). The vast majority of the mass of the aircra

In [7]:
df[cols].sample(10).head(10)

,text,text1,text2,text3
1092,"""\n\nps: Having an anonymous IP does not mean being """"everybody`s sockpuppet""""""",""" ps: Having an anonymous IP does not mean being """"everybody's sockpuppet""""""",""" ps: Having anonymous IP does not mean being """"everybody is sockpuppet""""""",p have anonymous ip do not mean be everybody be sockpuppet
332,"Adming is the one who abuses Wikipedia to force his stupid personal taste for just that one particular article. He is an admin and tries to make just one article out of line with wikipedia standards and style. Instead of bothering me, go ask that idiot why he tries to make Wikipedia articles inconsistent. 02:47, 27 Mar 2005 (UTC)","Adming is the one who abuses Wikipedia to force his stupid personal taste for just that one particular article. He is an admin and tries to make just one article out of line with wikipedia standards and style. Instead of bothering me, go ask that idiot why he tries to make Wikipedia articles inconsistent. 02:47, 27 Mar 2005 (UTC)","Adming is the one who abuses Wikipedia to force his stupid personal taste for just that one particular article. He is an admin and tries to make just one article out of line with wikipedia standards and style. Instead of bothering me, go ask that idiot why he tries to make Wikipedia articles inconsistent. 02:47, 27 Mar 2005 (UTC)",adming be the one who abuse wikipedia to force his stupid personal taste for just that one particular article he be an admin and try to make just one article out of line with wikipedia standard and style instead of bother i go ask that idiot why he try to make wikipedia article inconsistent 02 47 27 mar 2005 utc
10639,"""\nI don't know why are you so obsessed with Hindu script that you keep adding it to every article that comes in your way. Stop doing it. Stop trying to Indianize every Pakistan and Islam related article. Inida is not a Muslim country. It is a Hindu country. If your logic is followed, we need hundreds of transliterations of """"purdah"""" for every language of Purdah observing people. It's ridiculous. """,""" I don't know why are you so obsessed with Hindu script that you keep adding it to every article that comes in your way. Stop doing it. Stop trying to Indianize every Pakistan and Islam related article. Inida is not a Muslim country. It is a Hindu country. If your logic is followed, we need hundreds of transliterations of """"purdah"""" for every language of Purdah observing people. It's ridiculous. """,""" I do not know why are you so obsessed with Hindu script that you keep adding it to every article that comes in your way. Stop doing it. Stop trying to Indianize every Pakistan and Islam related article. Inida is not a Muslim country. It is a Hindu country. If your logic is followed, we need hundreds of transliterations of """"purdah"""" for every language of Purdah observing people. it is ridiculous. """,i do not know why be you so obsessed with hindu script that you keep add it to every article that come in your way stop do it stop try to indianize every pakistan and islam relate article inida be not a muslim country it be a hindu country if your logic be follow we need hundred of transliteration of purdah for every language of purdah observe people it be ridiculous
1799,"suck my dick first, maybe then i'll consider what you have to say.","suck my dick first, maybe then i'll consider what you have to say.","suck my dick first, maybe then I will consider what you have to say.",suck my dick first maybe then i will consider what you have to say
12123,"And go ahead and delete my account, I can't wait for it, I'm sick of having to check every retarded message you people post in my page!... If it gets deleted it gets permanently blanked!...","And go ahead and delete my account, I can't wait for it, I'm sick of having to check every retarded message you people post in my page!... If it gets deleted it gets permanently blanked!...","And go ahead and delete my account, I cannot wait for i

In [8]:
df[cols].sample(10).head(10)

,text,text1,text2,text3
9654,"could have fooled me, hens!\nlast time i checked you come from the low lands and have edited tape","could have fooled me, hens! last time i checked you come from the low lands and have edited tape","could have fooled me, hens! last time i checked you come from the low lands and have edited tape",could have fool i hen last time i check you come from the low land and have edit tape
736,"""\nI'm ignorant of these matters, but it looks like Germany has laws against denying the Holocaust or approving Nazi actions in a way that """"disturbs the peace"""", but not """"anti-Semitism"""" broadly drawn. I don't know whether German prosecutors are this unreasonable, but I know in the U.S. they have actually prosecuted children for making """"child pornography"""" of themselves ... if the comment actually is illegal to make there, I'm not sure it is safe even for a victim to keep reposting it. But Wikipedia itself should not be eager to gather up censorship laws from around the world.\nIt's too hard for me to puzzle through all the German text relevant to this case (which Google Translate handles far more poorly than Chinese or Arabic, alas). But I think as we're seeing in the present Fae ArbCom case, there's a huge divide in perceptions of bias between those who are members of an ethnic group, and those who are outside of it. Is it anti-gay to say that marriage should be between one man and one woman, or anti-Semitic to say that the U.S. should end all foreign aid to Israel? Or to keep a street named after a historical figure with a notable animosity to either group? Your answer depends on who you are. If we are to continue to have one Wikipedia for every nation and background, we'll have to be willing to accommodate that terms like """"homophobic"""" and """"anti-Semitic"""" mean very different things to very different people, and not punish anyone for using them the way they personally perceive them. """,""" I'm ignorant of these matters, but it looks like Germany has laws against denying the Holocaust or approving Nazi actions in a way that """"disturbs the peace"","" but not """"anti-Semitism"""" broadly drawn. I don't know whether German prosecutors are this unreasonable, but I know in the U.S. they have actually prosecuted children for making """"child pornography"""" of themselves ... if the comment actually is illegal to make there, I'm not sure it is safe even for a victim to keep reposting it. But Wikipedia itself should not be eager to gather up censorship laws from around the world. It's too hard for me to puzzle through all the German text relevant to this case (which Google Translate handles far more poorly than Chinese or Arabic, alas). But I think as we're seeing in the present Fae ArbCom case, there's a huge divide in perceptions of bias between those who are members of an ethnic group, and those who are outside of it. Is it anti-gay to say that marriage should be between one man and one woman, or anti-Semitic to say that the U.S. should end all foreign aid to Israel? Or to keep a street named after a historical figure with a notable animosity to either group? Your answer depends on who you are. If we are to continue to have one Wikipedia for every nation and background, we'll have to be willing to accommodate that terms like """"homophobic"""" and """"anti-Semitic"""" mean very different things to very different people, and not punish anyone for using them the way they personally perceive them. """,""" I am ignorant of these matters, but it looks like Germany has laws against denying the Holocaust or approving Nazi actions in a way that """"disturbs the peace"","" but not """"anti-Semitism"""" broadly drawn. I do not know whether German prosecutors are this unreasonable, but I know in the U.S. they have actually prosecuted children for making """"child pornography"""" of themselves ... if the comment actually is illegal to make there, I am not sure it is safe even for a victim to keep repos

# Review data

In [9]:
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
 1   text1   14251 non-null  object
 2   text2   14251 non-null  object
 3   text3   14251 non-null  object
dtypes: object(4)
memory usage: 445.5+ KB


In [10]:
%%time
df[cols].to_parquet("output/pre_val.parquet", index=False)

CPU times: user 109 ms, sys: 31.2 ms, total: 141 ms
Wall time: 159 ms
